# OrbitPy TAT-C Example for discrete time coverage calculations

OrbitPy is used to access calculations, followed by TAT-C to coduct coverage analysis.

In [36]:
from joblib import Parallel, delayed

import warnings

import json
from datetime import datetime, timedelta, timezone
from shapely.geometry import box, mapping
from scipy.stats import hmean
import pandas as pd

from astropy.time import Time as AstroPy_Time

from pydantic import AwareDatetime

from tatc.schemas import Instrument as TATC_Instrument, Satellite as TATC_Satellite, TwoLineElements, Point
from tatc.analysis import collect_orbit_track, OrbitCoordinate, OrbitOutput
from tatc.analysis import (
    collect_multi_observations,
    aggregate_observations,
    reduce_observations,
)

from orbitpy.util import OrbitState as OrbitPy_OrbitState, Spacecraft as OrbitPy_Spacecraft
from orbitpy.propagator import J2AnalyticalPropagator as OrbitPy_J2AnalyticalPropagator
from orbitpy.coveragecalculator import GridCoverage as OrbitPy_GridCoverage

from eose.orbits import GeneralPerturbationsOrbitState, Propagator
from eose.satellites import Satellite
from eose.utils import CartesianReferenceFrame
from eose.targets import TargetPoint
from eose.geometry import Position

from eose.access import (
    AccessSample,
    AccessRecord,
    AccessRequest,
    AccessResponse,
)
from eose.grids import UniformAngularGrid

from instrupy import Instrument as InstruPy_Instrument
from instrupy.basic_sensor_model import BasicSensorModel as InstruPy_BasicSensorModel

from eose.instruments import CircularGeometry, BasicSensor
from eose.datametrics import DataMetricsRequest, BasicSensorDataMetricsInstantaneous, DataMetricsSample, DataMetricsRecord, DataMetricsResponse

pd.set_option('display.max_rows', None)

### Define mission parameters

In [38]:
# define the orbit and the instrument
iss_omm_str = '[{"OBJECT_NAME":"ISS (ZARYA)","OBJECT_ID":"1998-067A","EPOCH":"2024-06-07T09:53:34.728000","MEAN_MOTION":15.50975122,"ECCENTRICITY":0.0005669,"INCLINATION":51.6419,"RA_OF_ASC_NODE":3.7199,"ARG_OF_PERICENTER":284.672,"MEAN_ANOMALY":139.0837,"EPHEMERIS_TYPE":0,"CLASSIFICATION_TYPE":"U","NORAD_CAT_ID":25544,"ELEMENT_SET_NO":999,"REV_AT_EPOCH":45703,"BSTAR":0.00033759,"MEAN_MOTION_DOT":0.00019541,"MEAN_MOTION_DDOT":0}]'
iss_omm = json.loads(iss_omm_str)[0]

basic_sensor = BasicSensor(   id="Atom",
                        mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        field_of_view = CircularGeometry(diameter=60.0),
                        data_rate= 10.5,
                        bits_per_pixel= 16
                    )

satellites=[
        Satellite(
            id="ISS",
            orbit=GeneralPerturbationsOrbitState.from_omm(iss_omm),
            payloads=[
                basic_sensor
            ]
        )
    ]

targets = UniformAngularGrid(
        delta_latitude=20, delta_longitude=20, region=mapping(box(-180, -50, 180, 50))
    ).as_targets()

mission_start = datetime(2024, 1, 1, tzinfo=timezone.utc)
mission_duration = timedelta(hours=2)
propagate_time_step = timedelta(minutes=0.5)

### Run access calculations with OrbitPy to get access-periods at Target ground points.

Access calculations also involve propagation calculations.


In [41]:
def access_tatc(request: AccessRequest) -> AccessResponse:
    if request.propagator != Propagator.J2:
        pass
    elif request.propagator != Propagator.SGP4:
        pass
    else:
        raise RuntimeError("OrbitPy only supports J2 and SGP4 propagators.")

    
    # Enumerate and convert from EOSE-API satellites to OrbitPy satellite objects. 
    # (Enumeration generates distinct orbit-instrument pairs for satellites equipped with multiple instruments.)
    OrbitPy_Satellites = []
    for satellite in request.satellites:
        for instru in satellite.payloads: 
            if instru.id in request.payload_ids:
                instru_type = instru.type
                if instru_type == "BasicSensor":
                    # TODO: Update to accept general instrument FOV geometries and orientation
                    instrupy_sensor = InstruPy_Instrument.from_dict({ "@type": "Basic Sensor",
                        "orientation": {"referenceFrame": "SC_BODY_FIXED", "convention": "REF_FRAME_ALIGNED"},
                        "fieldOfViewGeometry": {"shape": "CIRCULAR", "diameter": instru.field_of_view.diameter}
                    })
                else:
                    raise ValueError(f"{instru_type} instrument type is not supported. Only 'BasicSensor' instrument type is supported.")
                
                tle = satellite.orbit.to_tle()
                print(tle)

                orbit_state = OrbitPy_OrbitState.from_dict({"tle": {
                                                            "tle_line0": "Unknown",
                                                            "tle_line1": tle[0],
                                                            "tle_line2": tle[1]
                                                            }})
                
                sat = OrbitPy_Spacecraft(_id = satellite.id,
                                    orbitState = orbit_state,
                                    spacecraftBus = None, # spacecraft bus attributes are presently ignored in OrbitPy TODO: Consider the spacecraftbus orientation in OrbitPy
                                    instrument = [instrupy_sensor]
                                )
                warnings.warn("OrbitPy ignores spacecraft bus orientation, and assumes it to be aligned to the geocentric Nadir pointing frame.", UserWarning)
                
                print("\n", sat, "\n")
                OrbitPy_Satellites.append(sat)


                


    # run propagation with OrbitPy
    



In [ ]:
request = AccessRequest(
    satellites=satellites,
    targets=targets,
    start=mission_start,
    duration=mission_duration,
    propagator=Propagator.J2,
    payload_ids=["Atom"]
)

#display(request.model_dump_json())

access_response = access_tatc(request)

#display(access_response.model_dump_json())

#access_data = access_response.as_dataframe()

#display(access_data)

### Run coverage analysis with TAT-C on the Access response from OrbitPy